<a href="https://colab.research.google.com/github/noemicho/sentinela/blob/main/data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, load_img
import numpy as np

In [ ]:
# Função para criar o ImageDataGenerator para transformações específicas
def get_datagen(augmentation_type):
    if augmentation_type == "zoom":
        return ImageDataGenerator(zoom_range=0.4)
    elif augmentation_type == "rot":
        return ImageDataGenerator(rotation_range=45)
    elif augmentation_type == "saturation":
        return ImageDataGenerator(channel_shift_range=80.0)
    else:
        raise ValueError("Tipo de augmentação desconhecido!")

# Função de desfoque gaussiano
def apply_gaussian_blur(image, kernel_size=5):
    image = tf.image.resize(image, (128, 128))
    blurred_image = tf.nn.conv2d(
        input=tf.expand_dims(image, axis=0),
        filters=tf.random.normal([kernel_size, kernel_size, 3, 3]),
        strides=[1, 1, 1, 1],
        padding='SAME'
    )
    return tf.squeeze(blurred_image)


# Diretório de entrada e saída
input_dir = "/content/drive/MyDrive/Sentinela"
output_dir = "/content/drive/MyDrive/SentinelaTeste"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
for gesture_folder in os.listdir(input_dir):
    gesture_path = os.path.join(input_dir, gesture_folder)
    output_gesture_path = os.path.join(output_dir, gesture_folder)
    os.makedirs(output_gesture_path, exist_ok=True)

    # Renomear imagens no padrão
    original_images = [img for img in os.listdir(gesture_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
    for i, image_name in enumerate(original_images):
        image_path = os.path.join(gesture_path, image_name)
        new_name = f"{gesture_folder}_{i + 1:02d}"  # Novo nome: "paz_01"
        new_image_path = os.path.join(output_gesture_path, new_name + ".jpg")

        # Copiar a imagem original para a pasta de saída com o novo nome
        image = load_img(image_path)
        image.save(new_image_path)

        # Aplicar transformações específicas
        for augment_type in ["zoom", "rot", "saturation"]:
            datagen = get_datagen(augment_type)
            image_array = img_to_array(image)
            image_array = np.expand_dims(image_array, axis=0)

            # Gerar e salvar a nova imagem
            augmented_images = datagen.flow(image_array, batch_size=1)
            augmented_image = next(augmented_images)[0]
            augmented_pil = array_to_img(augmented_image)
            augmented_pil.save(os.path.join(output_gesture_path, f"{new_name}_{augment_type}.jpg"))

        # Aplicar desfoque manualmente
        blurred_image = apply_gaussian_blur(img_to_array(image))
        blurred_image_pil = array_to_img(blurred_image)
        blurred_image_pil.save(os.path.join(output_gesture_path, f"{new_name}_blur.jpg"))


print("Processamento concluído")


Processamento concluído
